# Enter an Activity Id and See a Summary of that Activity

In [1]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import ConfigManager
from garmindb.garmindb import GarminDb, Attributes, Device, ActivitiesDb, Activities, ActivityLaps, ActivityRecords, ActivitiesDevices
from idbutils import Location

from jupyter_funcs import format_number, format_string, format_temp, format_distance, linked_location
from maps import ActivityMap


activity_id = input('Enter the id of the activity you would like to display')

doc = snakemd.new_doc()

db_params_dict = ConfigManager.get_db_params()
garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]
altitude_units = unit_strings[fitfile.units.UnitTypes.altitude]
temp_units = unit_strings[fitfile.units.UnitTypes.tempurature]

activity = Activities.get(garmin_act_db, activity_id)

doc.add_heading(activity.name)
if activity.description:
    doc.add_paragraph(activity.description)

doc.add_heading("Summary", 3)
start_loc = Location(activity.start_lat , activity.start_long)
end_loc = Location(activity.stop_lat , activity.stop_long)
doc.add_table(
    [
        'Start', 'Id', 'Sport', 'Type', f'Distance ({distance_units})', "Ellapsed Time", "Moving Time", 'Avg HR', 'Max HR', 'Avg RR', 'Max RR', f'Gained ({altitude_units})', f'Lost ({altitude_units})',
        "Calories", f"Avg Temp (deg {temp_units})", 'Start Location', 'End Location'
    ],
    [
        [
            activity.start_time, activity_id, activity.sport, activity.type, format_distance(activity.distance), activity.elapsed_time, activity.moving_time, activity.avg_hr, activity.max_hr,
            format_number(activity.avg_rr), format_number(activity.max_rr), format_number(activity.ascent), format_number(activity.descent),
            activity.calories, format_temp(activity.avg_temperature), linked_location(start_loc), linked_location(end_loc)
        ]
    ]
)

doc.add_heading("Laps", 3)
laps = ActivityLaps.get_activity(garmin_act_db, activity_id)
lap_stats = []
for lap in laps:
    start_loc = Location(lap.start_lat , lap.start_long)
    end_loc = Location(lap.stop_lat , lap.stop_long)
    lap_stats.append(
        [
            lap.lap, format_distance(lap.distance), lap.elapsed_time, lap.moving_time, lap.avg_hr, lap.max_hr, format_number(lap.avg_rr), format_number(lap.max_rr),
            format_number(lap.ascent), format_number(lap.descent), lap.calories, format_temp(lap.avg_temperature), linked_location(start_loc), linked_location(end_loc)
        ]
    )
doc.add_table(
    [
        'Lap', f'Distance ({distance_units})', "Ellapsed Time", "Moving Time", 'Avg HR', 'Max HR', 'Avg RR', 'Max RR', f'Gained ({altitude_units})',
        f'Lost ({altitude_units})', "Calories", f"Avg Temp (deg {temp_units})", 'Start Location', 'End Location'
    ],
    lap_stats
)

doc.add_heading("Time In Heart Rate Zones", 3)
doc.add_table(
    ['Zone', 'HR Threshold', "Time"],
    [
        ["1", activity.hrz_1_hr, activity.hrz_1_time],
        ["2", activity.hrz_2_hr, activity.hrz_2_time],
        ["3", activity.hrz_3_hr, activity.hrz_3_time],
        ["4", activity.hrz_4_hr, activity.hrz_4_time],
        ["5", activity.hrz_5_hr, activity.hrz_5_time],
    ])

doc.add_heading("Training Effect", 3)
doc.add_table(
    ['Type', 'Rating'],
    [
        ["Aerobic", activity.training_effect],
        ["Anaerobic", activity.anaerobic_training_effect],
    ])

device_entries = ActivitiesDevices.get_activity(garmin_act_db, activity_id)
if len(device_entries):
    device_rows = []
    for device_entry in device_entries:
        device = Device.get(garmin_db, device_entry.device_serial_number)
        device_rows.append([device.manufacturer.name, device.product, device.device_type, device.serial_number, format_string(device.hardware_version)])
    doc.add_heading("Devices", 3)
    doc.add_table(['Manufacturer', 'Product', 'Type', 'Serial Number', 'Hardware Version'], device_rows)

display(Markdown(str(doc)))

if len(laps) and laps[0].start_lat is not None:
    records = ActivityRecords.get_activity(garmin_act_db, activity_id)
    if len(records) and records[-1].position_lat is not None:
        map = ActivityMap(records, laps)
        map.display()
    else:
        print(f"No record location data in {len(records)} records")
else:
    print("No lap location data")


AttributeError: 'NoneType' object has no attribute 'name'